In [1]:
from pathlib import Path
import os
import hydra
import torch

# Create a temporary directory to store the cache and experiment artifacts
# SAVE_DIR = Path(tempfile.gettempdir()) / 'tutorial_nuplan_framework'  # optionally replace with persistent dir
SAVE_DIR = Path('/home/nishka')
EXPERIMENT = 'training_logs/exp/training'
JOB_NAME = 'gpu0workerstrain_simple_vector_map_model'
LOG_DIR = SAVE_DIR / EXPERIMENT / JOB_NAME

# Get the checkpoint of the trained model
last_experiment = sorted(os.listdir(LOG_DIR))[-1]
train_experiment_dir = sorted(LOG_DIR.iterdir())[-1]
checkpoint = sorted((train_experiment_dir / 'checkpoints').iterdir())[-1]
MODEL_PATH = str(checkpoint)
MODEL_PATH_cfg = str(checkpoint).replace("=", "\=")
MODEL_PATH

'/home/nishka/training_logs/exp/training/gpu0workerstrain_simple_vector_map_model/2023.07.14.19.48.58/checkpoints/epoch=9.ckpt'

In [2]:
# Name of the experiment
EXPERIMENT = 'Simple_VMM_Experiment'

### Not used ###
CHALLENGE = 'open_loop_boxes'  # [open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents]
DATASET_PARAMS = [
    'scenario_builder=nuplan_mini',  # use nuplan mini database
    'scenario_filter=all_scenarios',  # initially select all scenarios in the database
    'scenario_filter.scenario_types=[near_multiple_vehicles, on_pickup_dropoff, starting_unprotected_cross_turn, high_magnitude_jerk]',  # select scenario types
    'scenario_filter.num_scenarios_per_type=10',  # use 10 scenarios per scenario type
]
# Location of path with all simulation configs
CONFIG_PATH = '../nuplan/planning/script/config/simulation'
CONFIG_NAME = 'default_simulation'
##################

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'experiment_name={EXPERIMENT}',
    f'group={SAVE_DIR}',
    'planner=ml_planner',
    'model=simple_vector_model',
    'planner.ml_planner.model_config=${model}',  # hydra notation to select model config
    f'planner.ml_planner.checkpoint_path={MODEL_PATH_cfg}',  # this path can be replaced by the checkpoint of the model trained in the previous section
    f'+simulation={CHALLENGE}',
    *DATASET_PARAMS,
])
planner_cfg = cfg.planner.ml_planner

In [3]:
from nuplan.planning.script.builders.model_builder import build_torch_module_wrapper
from nuplan.planning.training.modeling.lightning_module_wrapper import LightningModuleWrapper

torch_module_wrapper = build_torch_module_wrapper(planner_cfg.model_config)
model = LightningModuleWrapper.load_from_checkpoint(MODEL_PATH, model=torch_module_wrapper).model

In [4]:
model

VectorMapSimpleMLP(
  (vectormap_mlp): Sequential(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
  )
  (ego_mlp): Sequential(
    (0): Linear(in_features=15, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
  )
  (agent_mlp): Sequential(
    (0): Linear(in_features=40, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
  )
  (_mlp): Sequential(
    (0): Linear(in_features=384, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=48, bias=True)
  )
)

In [5]:
import pickle
from typing import cast
from nuplan.planning.training.preprocessing.features.agents import Agents
from nuplan.planning.training.preprocessing.features.trajectory import Trajectory
from nuplan.planning.training.preprocessing.features.vector_map import VectorMap
import numpy as np

with open('examples/agents', 'rb') as pick:
    agents = pickle.load(pick)
    agents_obj = Agents(ego=torch.Tensor(agents['ego']), agents=torch.Tensor(agents['agents']))

with open('examples/vector_map', 'rb') as pick:
    vmap = pickle.load(pick)
    vmap_obj = VectorMap(coords=torch.Tensor(vmap['coords']), 
                         lane_groupings=vmap['lane_groupings'],
                        multi_scale_connections=vmap['multi_scale_connections'],
                        on_route_status=vmap['on_route_status'],
                        traffic_light_data=vmap['traffic_light_data'])

# with open('examples/trajectory', 'rb') as pick:
#     traj = pickle.load(pick)

In [6]:
model.eval()

features = {'vector_map': vmap_obj, 'agents': agents_obj}

model.forward(features)

LOOK HERE
<class 'torch.Tensor'>
tensor([[ 4.6566e-10,  0.0000e+00,  2.1458e-06],
        [ 4.6566e-10,  0.0000e+00, -6.3777e-05],
        [ 4.6566e-10,  0.0000e+00, -5.1141e-05],
        [ 4.6566e-10,  0.0000e+00, -3.8862e-05],
        [ 4.6566e-10,  0.0000e+00, -2.7756e-17]])
<class 'torch.Tensor'>
tensor([[ 4.6566e-10,  0.0000e+00,  2.1458e-06,  4.6566e-10,  0.0000e+00,
         -6.3777e-05,  4.6566e-10,  0.0000e+00, -5.1141e-05,  4.6566e-10,
          0.0000e+00, -3.8862e-05,  4.6566e-10,  0.0000e+00, -2.7756e-17]])


{'trajectory': Trajectory(data=tensor([[[-1.9260e-01, -1.2288e-01,  6.0846e-02],
          [-8.9704e-02, -3.8206e-02, -2.5725e-02],
          [-1.7734e-01, -1.6475e-01, -3.7576e-02],
          [-2.4087e-01, -1.8800e-01, -6.3568e-03],
          [-1.9540e-01, -2.4304e-01,  2.8779e-03],
          [-3.2237e-01, -2.6313e-01,  2.3570e-02],
          [-3.0205e-01, -2.7913e-01, -1.7839e-02],
          [-1.9841e-01, -2.5822e-01,  6.7995e-03],
          [-4.2149e-01, -3.2694e-01, -4.8268e-02],
          [-1.7830e-01, -2.6053e-01, -4.0298e-02],
          [-1.4290e-01, -2.8508e-01, -8.0102e-02],
          [ 2.1691e-01, -2.2834e-01, -2.1272e-02],
          [ 3.9029e-01, -3.6305e-01, -1.7443e-03],
          [ 8.1347e-01, -4.4432e-01, -2.6769e-02],
          [ 1.3167e+00, -3.8198e-01, -3.9249e-02],
          [ 1.8378e+00, -3.9430e-01,  1.2404e-02]]], grad_fn=<ViewBackward>))}